### Simple Machine Learning

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier

from scipy.stats import loguniform, uniform  # for sampling continuous hyperparams
# loguniform ~ for parameters that vary over several orders of magnitude (e.g., C)
# uniform     ~ for parameters that vary over a linear scale

# ----------------------------------------------------
# Load your data
# ----------------------------------------------------
df = pd.read_csv('master.csv')
#
# The user says we have columns like:
#   participant_number, subexperiment_number, time_subexperiment, ...
#   feedback_score_subexperiment (the target), ...
# We'll drop participant_number, subexperiment_number
# We'll keep only the columns of interest

use_cols = [
    'time_subexperiment',
    'accuracy_subexperiment', 
    'accuracy_total',
    'answer_correct_subexperiment',
    'answer_incorrect_subexperiment',
    'feedback_score_subexperiment',  # target
    'trial_duration_1', 'trial_duration_2', 'trial_duration_3', 'trial_duration_4',
    'trial_duration_mean',
    'performance_subexperiment',
    'feedback_score_tutorial_1', 'feedback_score_tutorial_2', 'feedback_score_tutorial_3',
    'feedback_score_rest_1',
    'O2Hb_highest_peak', 'O2Hb_lowest_peak', 'O2Hb_average_peak', 'O2Hb_difference_peak',
    'O2Hb_auc',
    'O2Hb_highest_peak_trial_1', 'O2Hb_highest_peak_trial_2', 'O2Hb_highest_peak_trial_3', 'O2Hb_highest_peak_trial_4',
    'O2Hb_lowest_peak_trial_1', 'O2Hb_lowest_peak_trial_2', 'O2Hb_lowest_peak_trial_3', 'O2Hb_lowest_peak_trial_4',
    'O2Hb_average_peak_trial_1', 'O2Hb_average_peak_trial_2', 'O2Hb_average_peak_trial_3', 'O2Hb_average_peak_trial_4',
    'O2Hb_difference_peak_trial_1', 'O2Hb_difference_peak_trial_2', 'O2Hb_difference_peak_trial_3', 'O2Hb_difference_peak_trial_4',
    'O2Hb_auc_peak_trial_1', 'O2Hb_auc_peak_trial_2', 'O2Hb_auc_peak_trial_3', 'O2Hb_auc_peak_trial_4'
]

# Suppose df is your entire dataset:
df = df[ ['participant_number','subexperiment_number'] + use_cols ]
df = df.drop(['participant_number','subexperiment_number'], axis=1)

# Drop any NaNs if needed
df.fillna(0, inplace=True)

# Now define features X, and target y
X = df.drop('feedback_score_subexperiment', axis=1)
y = df['feedback_score_subexperiment']

# ----------------------------------------------------
# Quick check: classification or regression?
# ----------------------------------------------------
# If 'feedback_score_subexperiment' is categorical (like discrete classes 1..5),
# we do classification. If continuous, switch to regressor versions.

# We'll assume it's classification for this example.

# ----------------------------------------------------
# Model Pipelines
# ----------------------------------------------------
pipeline_lr = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(max_iter=2000))
])

pipeline_svm = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', SVC())
])

pipeline_lda = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LinearDiscriminantAnalysis())
])

pipeline_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', RandomForestClassifier())
])


# ----------------------------------------------------
# Hyperparameter distributions for RandomizedSearch
# ----------------------------------------------------
# We use 'loguniform' for parameters that can span orders of magnitude,
# e.g. C in logistic regression or SVM, because it can be anywhere from 1e-3 to 1e3.
# For RandomForest, we can sample # of estimators from a discrete set, etc.

param_dist_lr = {
    'clf__C': loguniform(1e-3, 1e3),        # from 0.001 to 1000
    'clf__penalty': ['l2'],
    'clf__solver': ['lbfgs', 'saga']        # saga can handle L1 but also L2
}

param_dist_svm = {
    'clf__C': loguniform(1e-3, 1e3),
    'clf__kernel': ['linear', 'rbf', 'poly'],
    'clf__gamma': loguniform(1e-4, 10)      # or 'scale', but let's search explicitly
}

param_dist_lda = {
    'clf__solver': ['svd', 'lsqr', 'eigen'],
    # For 'lsqr' or 'eigen', we can optionally search for shrinkage factor
    'clf__shrinkage': [None, 'auto', 0.0, 0.25, 0.5, 0.75, 1.0]
}

param_dist_rf = {
    'clf__n_estimators': [50, 100, 200, 500],
    'clf__max_depth': [None, 5, 10, 20, 50],
    'clf__min_samples_split': [2, 5, 10],
    'clf__min_samples_leaf': [1, 2, 5],
    'clf__max_features': ['sqrt', 'log2', None]  # controls feature subsampling
}


# ----------------------------------------------------
# Nested Cross-Validation Setup
# ----------------------------------------------------
# We create an outer StratifiedKFold for the final evaluation.
# For each outer fold, we'll do an inner RandomizedSearchCV to select hyperparams.
# We'll store the performance from each outer fold and average.

N_SPLITS_OUTER = 3  # e.g., 5 outer folds
N_SPLITS_INNER = 3  # e.g., 5 inner folds
N_ITER_SEARCH = 50  # number of random samples in hyperparameter search

outer_cv = StratifiedKFold(n_splits=N_SPLITS_OUTER, shuffle=True, random_state=42)

# We'll define a helper function that does nested CV for a pipeline & param distribution
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

def nested_cv_score(pipeline, param_dist, X, y, n_splits_outer, n_splits_inner, n_iter):
    """
    Perform nested CV with given pipeline & param distribution.
    Returns list of outer fold scores.
    """
    outer_cv = StratifiedKFold(n_splits=n_splits_outer, shuffle=True, random_state=42)
    outer_scores = []

    fold_idx = 1
    for train_idx, test_idx in outer_cv.split(X, y):
        X_train_outer, X_test_outer = X.iloc[train_idx], X.iloc[test_idx]
        y_train_outer, y_test_outer = y.iloc[train_idx], y.iloc[test_idx]

        # Inner cross-validation for hyperparameter tuning
        inner_cv = StratifiedKFold(n_splits=n_splits_inner, shuffle=True, random_state=fold_idx)

        random_search = RandomizedSearchCV(
            estimator=pipeline,
            param_distributions=param_dist,
            n_iter=n_iter,
            cv=inner_cv,
            scoring='accuracy',     # or f1, roc_auc, etc.
            random_state=fold_idx,
            n_jobs=-1,
            verbose=0
        )
        random_search.fit(X_train_outer, y_train_outer)

        # Best model from inner CV
        best_model = random_search.best_estimator_

        # Evaluate on the held-out test from the outer fold
        y_pred_outer = best_model.predict(X_test_outer)
        fold_acc = accuracy_score(y_test_outer, y_pred_outer)
        outer_scores.append(fold_acc)
        fold_idx += 1
    
    return outer_scores

# ----------------------------------------------------
# Run nested CV for each of the four models
# ----------------------------------------------------
models = [
    ("LogisticRegression", pipeline_lr, param_dist_lr),
    ("SVM", pipeline_svm, param_dist_svm),
    ("LDA", pipeline_lda, param_dist_lda),
    ("RandomForest", pipeline_rf, param_dist_rf),
]

results = {}
for name, pipe, params in models:
    print(f"\n=== Nested CV for {name} ===")
    scores = nested_cv_score(pipe, params, X, y,
                             n_splits_outer=N_SPLITS_OUTER,
                             n_splits_inner=N_SPLITS_INNER,
                             n_iter=N_ITER_SEARCH)
    mean_score = np.mean(scores)
    std_score = np.std(scores)
    print(f"{name} accuracy (outer CV): {mean_score:.3f} +/- {std_score:.3f}")
    results[name] = (mean_score, std_score)

print("\nAll results:", results)



=== Nested CV for LogisticRegression ===


C:\Users\Mobile Workstation 3\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(
C:\Users\Mobile Workstation 3\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(
C:\Users\Mobile Workstation 3\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


LogisticRegression accuracy (outer CV): 0.281 +/- 0.049

=== Nested CV for SVM ===


C:\Users\Mobile Workstation 3\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(
C:\Users\Mobile Workstation 3\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(
C:\Users\Mobile Workstation 3\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


SVM accuracy (outer CV): 0.256 +/- 0.014

=== Nested CV for LDA ===


C:\Users\Mobile Workstation 3\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 21 is smaller than n_iter=50. Running 21 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\Users\Mobile Workstation 3\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(
C:\Users\Mobile Workstation 3\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
24 fits failed out of a total of 63.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the f

LDA accuracy (outer CV): 0.279 +/- 0.039

=== Nested CV for RandomForest ===


C:\Users\Mobile Workstation 3\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(
C:\Users\Mobile Workstation 3\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


RandomForest accuracy (outer CV): 0.271 +/- 0.043

All results: {'LogisticRegression': (0.28125, 0.04868050602311634), 'SVM': (0.25625000000000003, 0.013501543121683054), 'LDA': (0.2791666666666666, 0.03897559777889522), 'RandomForest': (0.2708333333333333, 0.04340138886666596)}
